# データサイエンス特論2024: Lecture14 LLM時代のデータサイエンスとは(RAG)
## RAG(Retrieval Augmented Generation)の例


(C)2024 岩政 幹人
> 注：本試行を勝手に公開することを禁ず


ChatGPTに代表される、大規模言語モデル(LLM)を活用した様々なタスクの実現が可能になってきました。

ここでは、代表的なLLMの活用例として、特定の文章を対象にしたQ&Aを行うアプリケーションを想定します。

RAG:Retrieval Augmented Generationと呼ばれる、LLMの利用パターンになります。

以下ChatGPTの説明より、
> 情報検索を行うリトリーバー（Retriever）と、検索された情報を元に文章を生成するジェネレーター（Generator）が組み合わされます。
> 具体的には、リトリーバーは与えられた質問やテキストに関連する情報をデータベースや文書コレクションから検索し、その情報を抽出します。そして、ジェネレーターはリトリーバーが検索した情報を元に、より自然な形で回答や文章を生成します。

元となるテキストの取得については以下を参考にしました。

[LangChainを使ったRAGをElyza 7bを用いて試してみた](https://note.com/alexweberk/n/n3cffc010e9e9#98fafb11-8969-49b6-8990-fad204db936d)

また、OpenAIを有料利用していない方でも試せるように、オープンソースのLLMを使います。


>【注意】この環境は2024.5月に動作確認したものであり、利用するソフトのバージョンアップで使えなくなる可能性が高いことに注意。


オープンソースLLMを利用する方法として、ここでは、いったんローカル環境(といってもcolaboratory計算機ですが）にLLMをダウンロードして利用するローカルLLMを利用します。情報が外部に漏れることを気にする方はローカルLLMは便利ですね。ここではOllamaというソフトウエアを用いて、オープンソースのLLMをローカル環境にダウンロードして活用します。



###  利用環境
- [Ollama](https://ollama.com/)
- [llama_index](https://docs.llamaindex.ai/en/stable/)
- [langchain](https://python.langchain.com/docs/get_started/introduction) 、0.2.1

以下gemini様の説明より
> LlamaIndexとLangChainは、どちらも大規模言語モデル（LLM）を活用するための強力なライブラリですが、<略>
> LlamaIndexはQAシステムやデータインデックス作成に特化し、初心者でも扱いやすい簡潔さが魅力。一方、LangChainは汎用性の高いLLMフレームワークで、複雑なタスクや高度なカスタマイズにも対応。それぞれの強みを活かし、目的に合ったツールを選ぶことで、LLM活用の可能性が大きく広がります。

以下copilot様より、、
> Ollamaは、ユーザーがローカル環境で大規模な言語モデルを簡単に動作させることができるツールです。LLama2やCode Llamaなどのモデルを実行することができ、カスタマイズや独自のモデルの作成も可能です

## Ollamaをインストールします。


In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
!sudo apt install -y neofetch

colabマシンのスペックを見ます。

In [ ]:
!neofetch

### ollama serverを起動

colab環境にてollamaを使うためのサーバーを起動します。このサーバーは、外部から(pythonなどから）ollamaが管理するllmにアクセスるためのサーバーでもあります。

ローカルLLMを使っていて、時々以下のようなエラーが出る場合は、ollama severが停止している可能性があるので、↓のセルをもう一度起動しましょう。

> ConnectError: [Errno 99] Cannot assign requested address

In [ ]:
import subprocess
import time

# Start ollama as a backrgound process
command = "nohup ollama serve&"

# Use subprocess.Popen to start the process in the background
process = subprocess.Popen(command,
                            shell=True,
                           stdout=subprocess.PIPE,
                           stderr=subprocess.PIPE)
print("Process ID:", process.pid+1)
time.sleep(5)  # Makes Python wait for 5 seconds

ollama serveの稼働を確かめます

- エラーがでるならば↑を再起動
- 帰ってこないならば、


```
!kill  <↑のProcess ID>
```

の後で、ollama serveを起動します。



In [ ]:
#!kill 2137

In [ ]:
!ollama list

### ここでollamaでモデルをダウンロードしてrunします。


- 途中で落ちると、ollamaのプロセスが落ちるだけなので、そこだけ再起動すればよい、HDDにpullした結果が残っている？ので、再起動すると、すでにpullしたものはそのまま使えます。

```
# これはコードとして書式設定されます
ollama run mistral
ollama run llama3
ollama run phi3
```

8B(ビリオン、80億パラメター）モデルならば、Colab環境でどうにか動きます。

とりあえず、llama2が無難なのでこれを使います、他のllmも試すとよいと思います。

In [ ]:
OLLAMA_MODEL='gemma:7b'
OLLAMA_MODEL='mistral'
#OLLAMA_MODEL='phi3'
#OLLAMA_MODEL='wizardlm2'
OLLAMA_MODEL='llama3'
#OLLAMA_MODEL='llama2'

# Set it at the OS level
import os
os.environ['OLLAMA_MODEL'] = OLLAMA_MODEL
!echo $OLLAMA_MODEL

LLM(例えばllama3)をローカル環境(ここではColabのマシン)にダウンロードします。

In [ ]:
!ollama run $OLLAMA_MODEL "Explain AI in one line"

必要となるLangChainと関連ライブラリをインストールします。

In [ ]:
!pip install --quiet -U langchain langchainhub langchain_community

In [ ]:
# langchinのバージョンを確認します、動作したのは0.2.1 (2023.5.25時)
import langchain

print(langchain.__version__)

### 動作確認、Smoke Testです。

In [ ]:
from langchain_community.chat_models import ChatOllama

#OLLAMA_MODEL='mistral'
llm = ChatOllama(model=OLLAMA_MODEL)

llm.invoke("Who are you?")

## LangchainをつかったRAG

## まずはテキストを準備します

In [ ]:
!pip install trafilatura  sentence_transformers

In [ ]:
from trafilatura import fetch_url, extract
import os
os.makedirs("data", exist_ok=True)

url = "https://ja.m.wikipedia.org/wiki/ONE_PIECE"
filename = 'textfile.txt'

document = fetch_url(url)
text = extract(document)
print(text[:1000])

# 取ってきたテキストを一時的に保存
with open("data/"+filename, 'w', encoding='utf-8') as f:
    f.write(text)

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

loader = TextLoader("data/"+filename, encoding='utf-8')
documents = loader.load()

documents

In [ ]:
documents[0].page_content

ドキュメントを、分割します。これは。LLMには入力データの長さ（コンテキスト長）に制限があるので、Q&Aにおいても元の参照データを一度にコンテキスト情報として与えられないという制約があります。

そこで、chunkというサイズにテキストをバラバラにして、それをベクトル化データベースに格納します。

まずは、chunk_size=500で分割します。

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(documents)


In [ ]:
len(all_splits)

ちゃんと分割できてるかどうかを中身を見ます。

In [ ]:
all_splits[30:33]

In [ ]:
!pip install faiss-gpu

分割された文章の断片(chunk)を、ベクトル化データベース(ここではメタ社のFAISSを使います）に格納します。

In [ ]:
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

vectorstore = FAISS.from_documents(documents=all_splits, embedding=embeddings)

Ollamaをつかって、LLMのオブジェクト(ollama)を作成し、ベクトルデータベースからQ&Aを行うqachainを作っておきます。

In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import Ollama

ollama = Ollama(model=OLLAMA_MODEL)
qachain=RetrievalQA.from_chain_type(ollama, retriever=vectorstore.as_retriever(
    search_kwargs={"k": 3},
    ))

RAGは

1. ベクトルDBから質問に類似するchunkを類似検索により取得する
2. 取得されたchunk(複数）を、入力コンテキストに積んで、これに対する質問プロンプトを作成し、LLMを呼ぶ。
3. LLMは質問に対する回答を出力する

という動きをします。最初にベクトルＤＢから質問に対する類似のチャンクを得ます。

In [ ]:
question="ニコ・ロビンの職業は何ですか？"
question="エネルは何者ですか？"
#question="チョッパーの特殊能力は何ですか？"
#question="サンジは麦わらの一味に加わる前には何をしていたか？"
question="ワポルは何者であり、最後はどうなったか？"
docs = vectorstore.similarity_search(question)
docs

得られた結果から、質問に対する回答をＬＬＭが出力します。

In [ ]:
qachain.invoke({"query": question})

結果はどうでしょうか？↑のセルを起動するごとに異なる回答が生成されます。ＬＬＭによっては、出力が英語だったりします。

## llamaindexでRAGの実現

In [ ]:
!pip install llama-index llama-index-llms-ollama llama_index.embeddings.huggingface llama_index.readers.wikipedia

In [ ]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import display


Settings.llm =  Ollama(model=OLLAMA_MODEL, request_timeout=60.0)
Settings.embed_model = HuggingFaceEmbedding(
    #model_name="BAAI/bge-small-en-v1.5"
    model_name="intfloat/multilingual-e5-large"
)

In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex

# ドキュメントの読み込み
documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)

In [ ]:
query_engine = index.as_query_engine(
    similarity_top_k=3)

In [ ]:
question="ニコ・ロビンの職業は何ですか？"
#question="エネルは何者ですか？"
question="チョッパーの特殊能力は何ですか？"
#question="サンジは麦わらの一味に加わる前には何をしていたか？"
question="ワポルは何者であり、最後はどうなったか？"
response = query_engine.query(question)
display(response.response)

ベクトルＤＢから得られた類似チャンクを表示できます。

In [ ]:
for node in response.source_nodes:
  print(node.text)
  print(node.score)

プロンプトをカスタマイズできるらしい、

https://lightning.ai/lightning-ai/studios/compare-llama-3-and-phi-3-using-rag?utm_source=akshay

In [ ]:
from llama_index.core import PromptTemplate

qa_prompt_tmpl_str = (
            "コンテキスト情報は以下である.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "上記コンテキスト情報が与えられたとき、質問queryに対してステップbyステップに考察してAnswerを出力してほしい,出力は日本語で.\n"
            "Query: {query_str}\n"
            "Answer: "
            )

qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)
query_engine.update_prompts({"response_synthesizer:text_qa_template": qa_prompt_tmpl})

response = query_engine.query(question)
display(response.response)

## おまけ、日本語向けのLLM(ELYZA)を使う

いままでの例では、出力が日本語になりませんでした。そこで日本語向けに微調整された、モデルを使ってみます。

- ELYZAの4bit量子化モデル(コンパクト化されたモデル）を利用(gguf形式）
- ELYZAのModelfileは、llama2のものを利用
- OllamaにELYZAのggufモデルを読み込ませる。
- 普通に使う（モデル名を指定して）

こちらを参考にしています。

https://note.com/npaka/n/ndadbae6c6be5


In [ ]:
%%writefile Modelfile

FROM ./ELYZA-japanese-Llama-2-7b-instruct-q4_K_M.gguf
TEMPLATE """[INST] <<SYS>>{{ .System }}<</SYS>>

{{ .Prompt }} [/INST]
"""
PARAMETER stop "[INST]"
PARAMETER stop "[/INST]"
PARAMETER stop "<<SYS>>"
PARAMETER stop "<</SYS>>"

In [ ]:
!wget https://huggingface.co/mmnga/ELYZA-japanese-Llama-2-7b-instruct-gguf/resolve/main/ELYZA-japanese-Llama-2-7b-instruct-q4_K_M.gguf

In [ ]:
!ollama create elyza:7b-instruct -f Modelfile

ELYZAがollamaに認識されているか？

In [ ]:
!ollama list

In [ ]:
Settings.llm =  Ollama(model='elyza:7b-instruct', request_timeout=60.0)

In [ ]:
# ドキュメントの読み込み
#documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine(similarity_top_k=3)

In [ ]:
question="ニコ・ロビンの職業は何ですか？"
#question="エネルは何者ですか？"
#question="チョッパーの特殊能力は何ですか？"
#question="サンジは麦わらの一味に加わる前には何をしていたか？"
question="ワポルは何者であり、最後はどうなったか？"
response = query_engine.query(question)
display(response.response)

さらにプロンプトをカスタマイズしてみると。。

In [ ]:
from llama_index.core import PromptTemplate

qa_prompt_tmpl_str = (
            "コンテキスト情報は以下である.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "上記コンテキスト情報が与えられたとき、質問queryに対してステップbyステップに考察してAnswerを出力してほしい,出力は日本語で.\n"
            "Query: {query_str}\n"
            "Answer: "
            )

qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)
query_engine.update_prompts({"response_synthesizer:text_qa_template": qa_prompt_tmpl})

response = query_engine.query(question)
display(response.response)